Detailed description of run configuration could be found [here](../nablaDFT/README.md).

## Test example

In [ ]:
# model test example config
!cat ../config/gemnet-oc_test.yaml

In [ ]:
!python ../run.py --config-name gemnet-oc_test.yaml

## Inference on another dataset

For CLI-run please use example from the root of the repository:
```bash
python run.py --config-name gemnet-oc_predict.yaml
```

Detailed description could be found in [README](../nablaDFT/README.md)

In [ ]:
import os

import hydra
import pytorch_lightning as pl
import torch
from ase.db import connect
from nablaDFT import model_registry
from nablaDFT.dataset import PyGNablaDFTDataModule
from nablaDFT.pipelines import predict
from omegaconf import OmegaConf

### Paths and args

In [ ]:
data_args = {
    "root": "./datasets/nablaDFT/test",
    "dataset_name": "dataset_test_conformations_tiny",
    "batch_size": 4,
    "num_workers": 2,
}
if torch.cuda.is_available():
    accelerator = "gpu"
    devices = 1
else:
    accelerator = "cpu"
    devices = None

### Instantiate dataset and load model

In [ ]:
model = model_registry.get_pretrained_model("lightning", "GemNet-OC_train_large")
datamodule = PyGNablaDFTDataModule(**data_args)
trainer = pl.Trainer(accelerator=accelerator, devices=devices)
trainer.logger = False

In [ ]:
predict(trainer, model, datamodule, "GemNet-OC_train_large")

In [ ]:
db = connect("./predictions/GemNet-OC_train_large_dataset_test_conformations_tiny.db")

In [ ]:
len(db)

In [ ]:
row = db.get(1)

In [ ]:
energy, forces = row.data["energy_pred"], row.data["forces_pred"]

In [ ]:
print(f"Predicted energy: {energy}")
print("Predicted interatomic forces:")
print(forces)